In [1]:
# !pip install langchain langchain-community chromadb jq

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('menus.txt')
data = loader.load()

print(type(data))
print(len(data))

<class 'list'>
1


In [3]:
data

[Document(metadata={'source': 'menus.txt'}, page_content='2024-08-05의 조식메뉴는 쌀밥, 쇠고기감자국, 고등어캔무조림, 미니새송이볶음, 참나물무침, 배추김치이고 열량은 611칼로리입니다.\n2024-08-05의 중식메뉴는 쌀밥, 닭계장, 삼치튀김&타르소스, 우엉곤약조림, 치커리겉절이, 배추김치이고 열량은 623칼로리입니다.\n2024-08-05의 석식메뉴는 쌀밥, 어묵무국, 고추장불고기, 잡채, 콩나물무침, 배추김치이고 열량은 614칼로리입니다.\n2024-08-06의 조식메뉴는 쌀밥, 북어계란국, 돈장조림, 참치감자조림, 숙주나물, 배추김치이고 열량은 607칼로리입니다.\n2024-08-06의 중식메뉴는 쌀밥, 김치콩나물국, 탕수육&소스, 쭈꾸미볶음, 양상추샐러드, 배추김치이고 열량은 620칼로리입니다.\n2024-08-06의 석식메뉴는 쌀밥, 근대된장국, 닭봉무조림, 맛살팽이버섯전, 도라지오이무침, 배추김치이고 열량은 615칼로리입니다.\n2024-08-07의 조식메뉴는 쌀밥, 쇠고기버섯국, 임연수무조림, 어묵채볶음, 콩나물무침, 배추김치이고 열량은 610칼로리입니다.\n2024-08-07의 중식메뉴는 쌀밥, 북어미역국, 고추장불고기, 두부계란전&양념장, 숙주나물, 배추김치이고 열량은 616칼로리입니다.\n2024-08-07의 석식메뉴는 쌀밥, 감자채계란국, 쇠고기불고기, 도토리묵&파양념장, 양배추부추생채, 배추김치이고 열량은 608칼로리입니다.\n2024-08-08의 조식메뉴는 쌀밥, 솎음배추된장국, 돈육김치찜, 가지볶음, 무채볶음, 배추김치이고 열량은 612칼로리입니다.\n2024-08-08의 중식메뉴는 쌀밥, 콩나물국, 안동찜닭, 새송이버섯전, 도라지오이무침, 배추김치이고 열량은 619칼로리입니다.\n2024-08-08의 석식메뉴는 쌀밥, 해물순두부찌개, 제육볶음, 느타리버섯들깨볶음, 참나물무침, 배추김치이고 열량은 618칼로리입니다.\n2024-08-09의 조식메뉴는 쌀밥, 어묵곤약국, 돈장조림,

In [4]:
from dotenv import load_dotenv
import os
import json
import sys
import time
from tqdm import tqdm
# from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings

# Load environment variables
load_dotenv()

# Load API keys from environment variables
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')

start_time = time.time()

# Use the recursive character splitter
recur_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=60,
    separators=["\n"]
)

# Perform the splits using the splitter
data_splits = list(tqdm(recur_splitter.split_documents(data), desc="Splitting documents", unit="chunk"))
print(f"Number of splits: {len(data_splits)}")

# Vector Store 구축
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)

persist_directory = '.cache/db/menu'

vectordb = Chroma.from_documents(
    documents=data_splits, # 위에서 처리한 데이터 
    embedding=embeddings, # upstage solar embedding 1 large
    persist_directory=persist_directory)

vectordb.persist()
vectordb = None

end_time = time.time()
elapsed_time = end_time - start_time

Splitting documents: 100%|██████████| 20/20 [00:00<00:00, 229196.94chunk/s]


Number of splits: 20


/Users/seongyeon/Desktop/Nursing-home/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings


In [16]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE="""

# 당신의 역할
- 당신은 요양병원에서 환자들의 보호자에게 오늘의 식단을 알려주는 시스템입니다.

------
    
# 참고 사항
- 하루의 식사는 조식, 중식, 석식으로 구성된다.
- 하루 식단의 열량은 그 날의 조식, 중식, 석식을 더한 값이다.

------

# 지시 사항
- 조식, 중식, 석식이라는 단어보다는 아침, 점심, 저녁을 사용해서 답변해줘

------

# 식단표 및 원산지 : \n\t{context}

------

# 보호자의 질문 : \n\t{question}

------

# 중요
- 한국어로 답변해주세요
- 답변 뒤에 '본 식단은 식자재 수급 상황에 따라 다소 변경 될 수 있습니다.'를 추가해주세요.

# 답변 :
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [17]:
query = "오늘 2024년 7월 1일 메뉴 알려줘"
print(qa_chain.invoke(query))

오늘 2024년 7월 1일 메뉴는 아침 식사로 쌀밥, 북어계란국, 고추장불고기, 새송이버섯전, 치커리겉절이, 배추김치이고 열량은 615칼로리입니다. 점심 식사로 쌀밥, 건새우미역국, 안동찜닭, 새우까스&콘마요소스, 양배추부추무침, 배추김치이고 열량은 628칼로리입니다. 저녁 식사로 쌀밥, 김치찌개, 코다리강정, 계란장조림, 참나물무침, 배추김치이고 열량은 620칼로리입니다. 식단은 식자재 수급 상황에 따라 다소 변경될 수 있습니다.


In [18]:
query = "오늘 2024년 7월 1일 전체 칼로리 알려줘"
print(qa_chain.invoke(query))

2024년 7월 1일 전체 칼로리는 아침 605칼로리, 점심 628칼로리, 저녁 620칼로리입니다. 총 칼로리는 1853칼로리입니다.

본 식단은 식자재 수급 상황에 따라 다소 변경될 수 있습니다.


In [19]:
query = "김치는 원산지가 어떻게 돼?"
print(qa_chain.invoke(query))

김치 원산지 정보:
알타리김치: 알타리무(국내산), 고춧가루(국내산)
깍두기, 석박지: 무(국내산), 고춧가루(국내산)

본 식단은 식자재 수급 상황에 따라 다소 변경될 수 있습니다.
